In [ ]:
#  Gerekli kütüphaneler
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import os



# Google Drive'da model dosyalarının kaydedileceği klasör
model_dir = '/content/drive/MyDrive/AlzheimerModels'
os.makedirs(model_dir, exist_ok=True)

# Dataset klasörleri (hazır bölünmüş)
train_dir = '/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/train'
val_dir = '/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/val'
test_dir = '/content/drive/MyDrive/Alzheimer_dataset/Splitted_OriginalDataset/test'

#  Data augmentation ve preprocessing
batch_size = 128
img_size = (176, 208)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

from tensorflow.keras import backend as K
K.clear_session()

# DenseNet121 tabanlı model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(176, 208, 3))
base_model.trainable = True
for layer in base_model.layers[:20]:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='softmax')
])

model.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-5),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Callbacks
tensorboard_cb = TensorBoard(log_dir='/content/logs_densenet121')
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr_cb = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

best_model_path = os.path.join(model_dir, 'best_model_densenet.keras')

checkpoint_cb = ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

#  Class weights (dengesiz sınıf için)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

#  Modeli eğit
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    class_weight=class_weights,
    callbacks=[tensorboard_cb, early_stopping_cb, reduce_lr_cb, checkpoint_cb]
)

#  En iyi model yüklenip final olarak kaydedilir
final_model_path = os.path.join(model_dir, 'final_model_densenet.keras')
best_model = load_model(best_model_path)
best_model.save(final_model_path)

print(f"\n Eğitim tamamlandı ve model Google Drive'a kaydedildi: {final_model_path}")


Found 4479 images belonging to 4 classes.
Found 960 images belonging to 4 classes.
Found 961 images belonging to 4 classes.
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 38s/step - accuracy: 0.3130 - loss: 1.6591 
Epoch 1: val_loss improved from inf to 1.19726, saving model to /content/drive/MyDrive/AlzheimerModels/best_model_densenet.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 1957s 51s/step - accuracy: 0.3140 - loss: 1.6567 - val_accuracy: 0.4208 - val_loss: 1.1973 - learning_rate: 1.0000e-04
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4168 - loss: 1.1635
Epoch 2: val_loss improved from 1.19726 to 1.16594, saving model to /content/drive/MyDrive/AlzheimerModels/best_model_densenet.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 70s 2s/step - accuracy: 0.4177 - loss: 1.1613 - val_accuracy: 0.5354 - val_loss: 1.1659 - learning_rate: 1.0000e-04
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4915 - loss: 0.9647
Epoch 3: val_loss improved from 1.16594 to 1.05633, saving model to /content/drive/MyDrive/AlzheimerModels/best_model_densenet.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.4918 - loss:

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

#  Sınıf isimleri
class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']



model = load_model("/content/drive/MyDrive/AlzheimerModels/best_model_densenet.keras")

# Tahmin fonksiyonu
def predict(img):
    img = img.resize((208, 176))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    preds = model.predict(img_array)[0]
    predicted_class = class_names[np.argmax(preds)]
    confidence = np.max(preds)

    return f"{predicted_class} ({confidence:.2%})"

#  Gradio arayüzü
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="🧠 Alzheimer MRI Tahmin Arayüzü (DenseNet121)",
    description="Bir MRI görüntüsü yükleyin ve modelin tahminini görün."
)

#  Arayüzü başlat
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b277197776eee35f71.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
